In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

import xgboost as xgb
from sklearn.metrics import accuracy_score

from sklearn import preprocessing

from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import cross_validate
from sklearn.metrics import recall_score

from sklearn.ensemble import RandomForestClassifier

In [3]:
df = pd.read_csv('df.csv')
df.head()

,Unnamed: 0,Type,Name,Age,Breed1,Breed2,Breed2_new,Gender,Color1,Color2,...,Group,Fee,State,RescuerID,VideoAmt,Description,Length_Description,PetID,PhotoAmt,AdoptionSpeed
0,0,2,Nibble,3,299,0,0,1,1,7,...,0,100,41326,8480853f516546f6cf33aa88cd76c379,0,Nibble is a 3+ month old ball of cuteness. He ...,359,86e1089a3,1.0,2
1,1,2,No Name Yet,1,265,0,0,1,1,2,...,0,0,41401,3082c7125d8fb66f7dd4bff4192c8b14,0,I just found it alone yesterday near my apartm...,118,6296e909a,2.0,1
2,2,1,Brisco,1,307,0,0,1,2,7,...,0,0,41326,fa90fa5b1ee11c86938398b60abc32cb,0,"""Their pregnant mother was dumped by her irres...",395,3422e4906,7.0,3
3,3,1,Miko,4,307,0,0,2,1,2,...,0,150,41401,9238e4f44c71a75282e62f7136c6b240,0,"""Good guard dog, very alert, active, obedience...",148,5842f1ff5,8.0,2
4,4,1,Hunter,1,307,0,0,1,1,0,...,0,0,41326,95481e953f8aed9ec3d16fc4509537e8,0,"""This handsome yet cute boy is up for adoption...",392,850a43f90,3.0,2


In [4]:
for i in range(0,len(df.Name)):
    if df.Name.str.len()[i] <3:
        df.Name[i] = 'Meaningless'
        
df['Name'] = df['Name'].replace(['No Name', 'No Name Yet', 'None'], 'No_Name')
df['Meaningless_Name'] = np.where(np.logical_or(df['Name'] == 'No_Name', df['Name'] == 'Meaningless'), 1,0)
list_names = ['No_Name', 'Meaningless']
df['Has_Name'] = np.where(df['Name'].isin(list_names) == False, 1,0)
df = df.drop(['Name'],axis=1)
df.head()

C:\Users\colla\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Unnamed: 0,Type,Age,Breed1,Breed2,Breed2_new,Gender,Color1,Color2,Color3,...,State,RescuerID,VideoAmt,Description,Length_Description,PetID,PhotoAmt,AdoptionSpeed,Meaningless_Name,Has_Name
0,0,2,3,299,0,0,1,1,7,0,...,41326,8480853f516546f6cf33aa88cd76c379,0,Nibble is a 3+ month old ball of cuteness. He ...,359,86e1089a3,1.0,2,0,1
1,1,2,1,265,0,0,1,1,2,0,...,41401,3082c7125d8fb66f7dd4bff4192c8b14,0,I just found it alone yesterday near my apartm...,118,6296e909a,2.0,1,1,0
2,2,1,1,307,0,0,1,2,7,0,...,41326,fa90fa5b1ee11c86938398b60abc32cb,0,"""Their pregnant mother was dumped by her irres...",395,3422e4906,7.0,3,0,1
3,3,1,4,307,0,0,2,1,2,0,...,41401,9238e4f44c71a75282e62f7136c6b240,0,"""Good guard dog, very alert, active, obedience...",148,5842f1ff5,8.0,2,0,1
4,4,1,1,307,0,0,1,1,0,0,...,41326,95481e953f8aed9ec3d16fc4509537e8,0,"""This handsome yet cute boy is up for adoption...",392,850a43f90,3.0,2,0,1


In [5]:
rescuer_count = df.groupby(['RescuerID'])['PetID'].count().reset_index()
rescuer_count.columns = ['RescuerID', 'RescuerID_count']

# Merge as another feature onto main DF:
df = df.merge(rescuer_count, how='left', on='RescuerID')

In [6]:
petID= df['PetID']

In [7]:
df= df.drop(['RescuerID','PetID'], axis=1)

In [8]:
state_gdp = {
    41336: 116.679,
    41325: 40.596,
    41367: 23.02,
    41401: 190.075,
    41415: 5.984,
    41324: 37.274,
    41332: 42.389,
    41335: 52.452,
    41330: 67.629,
    41380: 5.642,
    41327: 81.284,
    41345: 80.167,
    41342: 121.414,
    41326: 280.698,
    41361: 32.270
}

state_population = {
    41336: 33.48283,
    41325: 19.47651,
    41367: 15.39601,
    41401: 16.74621,
    41415: 0.86908,
    41324: 8.21110,
    41332: 10.21064,
    41335: 15.00817,
    41330: 23.52743,
    41380: 2.31541,
    41327: 15.61383,
    41345: 32.06742,
    41342: 24.71140,
    41326: 54.62141,
    41361: 10.35977
}

df["state_gdp"] = df['State'].map(state_gdp)
df["state_population"] = df['State'].map(state_population)

In [9]:
categorical_columns = ['Breed1', 'Breed2','State','Color1','Color2','Color3']
for i in categorical_columns:
    df.loc[:, i] = pd.factorize(df.loc[:, i])[0]

In [10]:
df = df[['Type', 'Age', 'Breed1', 'Breed2', 'Gender', 'Color1',
       'Color2', 'Color3', 'Single_Colored', 'Double_Colored',
       'Triple_Colored', 'MaturitySize', 'FurLength', 'Vaccinated', 'Dewormed',
       'Sterilized', 'Health', 'Quantity', 'Group', 'Fee', 'State', 'VideoAmt',
       'Length_Description', 'PhotoAmt', 'Meaningless_Name',
       'Has_Name', 'RescuerID_count',
       'state_gdp', 'state_population', 'AdoptionSpeed']]

In [11]:
df2 = df.copy()
array1 = df.values
X_array1 = array1[:, 0:-1]
y_array1 = array1[:, -1]

In [12]:
def evaluate_classifiers(X, y, n_splits=4, shuffle=True, random_state=0):

    num_trees=100
    num_features=3
    seed= 0

    features = [('Normal',       X)]

    models = [('LogisticRegression',         LogisticRegression(solver="liblinear")),
              ('LinearDiscriminantAnalysis', LinearDiscriminantAnalysis()),
              ('KNeighborsClassifier',       KNeighborsClassifier()),
              ('Naive Bayes',                GaussianNB()),
              ('DecisionTreeClassifier',     DecisionTreeClassifier()),
              ('SupportVectorMachine',       SVC(gamma="scale")),
              ('DecisionTree',               DecisionTreeClassifier(random_state=seed)),
              ('RandomForest',               RandomForestClassifier(n_estimators=num_trees, max_features=num_features, random_state=seed)),
              ('XGBoost',                    xgb.XGBClassifier())]

    kfold = KFold(n_splits=n_splits, shuffle=shuffle, random_state=random_state)

    results = []
    for model in models:

        for feature in features:
            res = cross_val_score(model[1], feature[1], y, cv=kfold)

            [results.append((model[0], feature[0], model[0]+'_'+feature[0], r)) for r in res]

    results = pd.DataFrame(results, columns=['Model', 'FeatureScaling', 'Combined', 'Result'])
    return results

In [13]:
results_1 = evaluate_classifiers(X_array1,y_array1)
accuracy_1 = results_1.groupby(['Combined'],sort=False).mean().reset_index()
accuracy_1

C:\Users\colla\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\colla\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\colla\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\colla\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\colla\Anaconda3\lib\site-pa

,Combined,Result
0,LogisticRegression_Normal,0.367487
1,LinearDiscriminantAnalysis_Normal,0.365619
2,KNeighborsClassifier_Normal,0.336535
3,Naive Bayes_Normal,0.359149
4,DecisionTreeClassifier_Normal,0.370889
5,SupportVectorMachine_Normal,0.331465
6,DecisionTree_Normal,0.367887
7,RandomForest_Normal,0.449203
8,XGBoost_Normal,0.430258


In [14]:
#package for ensemble
from sklearn.ensemble import VotingClassifier

resall=pd.DataFrame()
res_w1=pd.DataFrame()

num_trees=100
num_features=3
seed=0

estimators=[]

#model 1 = xgboost
model1=xgb.XGBClassifier()
estimators.append(("xgb", model1))

#model 2 = random forest
model2=RandomForestClassifier(n_estimators=num_trees, max_features=num_features, random_state=seed)
estimators.append(("rf", model2))

model=VotingClassifier(estimators)

kfold=KFold(n_splits=10, random_state=seed)

results=cross_val_score(model, X_array1, y_array1, cv=kfold)

print(f'Voting Ensemble (xgb,rf) - Accuracy {results.mean()*100:.3f}% std {results.std()*100:3f}')

res_w1["Res"]=results
res_w1["Type"]="Voting"

resall=pd.concat([resall,res_w1], ignore_index=True)

Voting Ensemble (xgb,rf) - Accuracy 44.340% std 1.089745


In [15]:
feat_imp = pd.DataFrame(columns = ['name', 'xgb_importance','rf_importance'])

#xgb and rf models
model_rf=RandomForestClassifier(n_estimators=num_trees, max_features=num_features, random_state=seed)
model_rf.fit(X_array1,y_array1)
xgb_model = xgb.XGBClassifier()
xgb_model.fit(X_array1,y_array1)

#calculating xgb and rf feature importance then adding them to total column
for name, importance, importances in zip(df.columns, xgb_model.feature_importances_, model_rf.feature_importances_):
    feat_imp = feat_imp.append({'name':name, 'xgb_importance':importance, 'rf_importance':importances}, ignore_index = True)
    feat_imp['total'] = feat_imp['xgb_importance']+ feat_imp['rf_importance']
    feat_imp = feat_imp.sort_values('total',ascending=False)
    
feat_imp

,name,xgb_importance,rf_importance,total
0,RescuerID_count,0.091589,0.099633,0.191222
1,Age,0.095743,0.092004,0.187747
2,Length_Description,0.021525,0.130493,0.152019
3,Breed1,0.085497,0.055316,0.140813
4,PhotoAmt,0.048629,0.088803,0.137433
5,Sterilized,0.071322,0.025951,0.097273
6,Type,0.078966,0.015523,0.094489
28,state_population,0.052988,0.022163,0.075151
7,Color1,0.022008,0.053142,0.075150
8,Color2,0.020458,0.047759,0.068216


In [16]:
df.to_csv('df2.csv')

In [ ]:
from sklearn import model_selection
test_size = 0.2
seed = 0
X_train, X_test, Y_train, Y_test = train_test_split(X_array1, y_array1, test_size=test_size, random_state=seed)
# Fit the model on training set
model = RandomForestClassifier(n_estimators=1000, max_features=3, random_state=seed)
model.fit(X_train, Y_train)

import pickle
filename = 'best_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
y_pred = model.predict(X_test)

accuracy = accuracy_score(Y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt

conf_mat = confusion_matrix(y_true=Y_test, y_pred=y_pred)
print('Confusion matrix:\n', conf_mat)

labels = ['1', '2','3', '4']
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(conf_mat, cmap=plt.cm.Blues)
fig.colorbar(cax)
ax.set_xticklabels([''] + labels)
ax.set_yticklabels([''] + labels)
plt.xlabel('Predicted')
plt.ylabel('Expected')
plt.show()

In [ ]:
# Class count
count_class_1, count_class_2, count_class_3, count_class_4 = df.AdoptionSpeed.value_counts()

# Divide by class
df_class_1 = df[df['AdoptionSpeed'] == 1]
df_class_2 = df[df['AdoptionSpeed'] == 2]
df_class_3 = df[df['AdoptionSpeed'] == 3]
df_class_4 = df[df['AdoptionSpeed'] == 4]

In [ ]:
df_class_1_under = df_class_1.sample(count_class_4)
df_class_2_under = df_class_2.sample(count_class_4)
df_class_4_under = df_class_4.sample(count_class_4)

df_test_under = pd.concat([df_class_1_under, df_class_2_under, df_class_4_under, df_class_3], axis=0)

print('Random under-sampling:')
print(df_test_under.AdoptionSpeed.value_counts())

df_test_under.AdoptionSpeed.value_counts().plot(kind='bar', title='Count (Adoption Speed)');

In [ ]:
under = df_test_under.values
X_us = under[:, 0:-1]
y_us = under[:, -1]

In [ ]:
test_size = 0.2
seed = 0
X_train, X_test, Y_train, Y_test = train_test_split(X_us, y_us, test_size=test_size, random_state=seed)
# Fit the model on training set
model = RandomForestClassifier(n_estimators=1000, max_features=3, random_state=seed)
model.fit(X_train, Y_train)

filename = 'best_model_undersample.sav'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(Y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt

conf_mat = confusion_matrix(y_true=Y_test, y_pred=y_pred)
print('Confusion matrix:\n', conf_mat)

labels = ['1', '2','3', '4']
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(conf_mat, cmap=plt.cm.Blues)
fig.colorbar(cax)
ax.set_xticklabels([''] + labels)
ax.set_yticklabels([''] + labels)
plt.xlabel('Predicted')
plt.ylabel('Expected')
plt.show()

In [ ]:
df['PetID'] = petID

In [ ]:
df.to_csv('df_for_Rolfito.csv')